# Vector Store

In [10]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
load_dotenv()
google_embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
model  = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')




In [16]:
import time
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key='pcsk_6xp3m6_RZNgC8S9uU9hYcS48M8v6wF3pJbeQoRd37kmKtSBb7aNEwgfLn4dKGGEYAjTVDC')
index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [17]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=google_embedding)

In [18]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['4ff6ebdf-29c0-4ab5-89f0-924ff93a301b',
 'ce28c73c-329a-449c-9259-000e8e8fa681',
 'c5ecb699-ff15-4203-9751-b12d4e211da0',
 '3cf3e483-e0a0-483b-8481-6ba564e25bca',
 'db16947d-4ee7-4904-99df-6d5df76c419d',
 '42cedb2e-7cdf-43fb-9aed-d4ef6433064c',
 '2865601c-beff-47d4-baca-f784c7e0c0ff',
 'ed8ec5af-810d-4c6f-b139-065ec64bfce2',
 'd6293463-4742-45e9-acc9-de6bbbf6b307',
 '287950d4-8ef7-4b1c-b392-0a94aadd5e89']

In [19]:
results = vector_store.similarity_search(
    "What is weatherr forecast for tommorow",
    k=2,
   
)
print(results)

[Document(id='ce28c73c-329a-449c-9259-000e8e8fa681', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'), Document(id='287950d4-8ef7-4b1c-b392-0a94aadd5e89', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]


In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('../langchain-tutorial/documents/ragdocs.pdf')
docs = loader.load()



In [26]:
vector_store.add_documents(documents=docs)

['76221761-17b8-4179-b034-91b90ef3d448',
 '0452293b-0df3-4971-9a50-72c626af88b3',
 'c0ed0412-bfed-46fd-992f-36e86975bca5']

In [48]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
template = PromptTemplate(
    template='anser the give questtion {question} , based on provided information->{info}',
    input_variables=['question','info']
)

chain = template|model|parser
txt = "who are from charusat universsity"
results = vector_store.similarity_search(
    txt,
    k=3,
   
)

chain.invoke({'question':txt ,'info':results})

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 09 Apr 2025 05:40:34 GMT', 'Content-Type': 'text/plain', 'Content-Length': '12', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Malformed domain', 'www-authenticate': 'Malformed domain', 'server': 'envoy'})
HTTP response body: Unauthorized
